# Data collection 5 minute intervals

Gather public SPP Weis data from https://marketplace.spp.org/groups/operational-data-weis

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import pandas as pd

# logging
import logging

# define log
logging.basicConfig(level=logging.INFO)

# Set the logging level for the 'py4j' logger to ERROR or WARNING
logging.getLogger("py4j").setLevel(logging.ERROR) 

log = logging.getLogger(__name__)

In [ ]:
# set env vars
try:
    os.environ['AWS_ACCESS_KEY_ID'] = dbutils.secrets.get(scope = "aws", key = "AWS_ACCESS_KEY_ID")
    os.environ['AWS_SECRET_ACCESS_KEY'] = dbutils.secrets.get(scope = "aws", key = "AWS_SECRET_ACCESS_KEY")
except:
    print('not on DBX')

from dotenv import load_dotenv
load_dotenv()

In [ ]:
os.chdir('../..')
sys.path.append('./src')

In [ ]:
import src.data_collection as dc

## LMP settlement location prices

![_](../../imgs/lmp_settlement_location.PNG)

## LMP upsert

### 5 minute LMP interval files

In [ ]:
# getting ending time for data gathering
end_ts = (
    pd.Timestamp.utcnow().tz_convert("America/Chicago").tz_localize(None) - 
    pd.Timedelta('2D')
).tz_localize(None)

end_ts 

In [ ]:
range_df = dc.get_range_data_interval_5min_lmps(end_ts=end_ts,  n_periods=24*12)
range_df

In [ ]:
parquet_files = sorted([pf for pf in range_df if pf.endswith('.parquet')])
parquet_files

In [ ]:
dc.upsert_lmp(parquet_files)